# Million Rows Data Analysis

## Part I: Separating broad categories, Gender based behavior

Preliminaries, as usual.

In [ ]:
import os
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



# remove warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
# ---

pd.options.display.max_columns= 2000

%matplotlib inline

In [ ]:
data=pd.read_csv("1M.csv")

In [ ]:
print data.shape
#print data.dtypes
data.p2_liked.describe()

### Removing useless columns

In [ ]:
useless_columns=['answeredFromTelegram', 'confirmed',
                   'p1_timezone', 'p2_timezone', 'unmatched']
data.drop(useless_columns, axis=1, inplace=True)

### Handeling [p2_liked]
Since these rows are contain two sets of data, let us duplicate them, and then remerge them with main data after switching [p1\_...] and [p2\_...] data. That will make [p2_liked] useless and we will delete it.

In [ ]:
datap2=data[data.p2_liked>=0]
datap2.shape

In [ ]:
print sorted(list(data))

In [ ]:
p2_cols={'p1_age' :  'p2_age' ,
 'p1_country' :  'p2_country' ,
 'p1_education' :  'p2_education' ,
 'p1_ethnicity' :  'p2_ethnicity' ,
 'p1_flavor' :  'p2_flavor' ,
 'p1_gender' :  'p2_gender' ,
 'p1_id' :  'p2_id' ,
 'p1_languages' :  'p2_languages' ,
 'p1_liked' :  'p2_liked' ,
 'p1_premium' :  'p2_premium' ,
 'p1_religiosity' :  'p2_religiosity' ,
 'p2_age' :  'p1_age' ,
 'p2_country' :  'p1_country' ,
 'p2_education' :  'p1_education' ,
 'p2_ethnicity' :  'p1_ethnicity' ,
 'p2_flavor' :  'p1_flavor' ,
 'p2_gender' :  'p1_gender' ,
 'p2_id' :  'p1_id' ,
 'p2_languages' :  'p1_languages' ,
 'p2_liked' :  'p1_liked' ,
 'p2_premium' :  'p1_premium' ,
 'p2_religiosity' :  'p1_religiosity' }
datap2n= datap2.rename(columns=p2_cols)

In [ ]:
data2 = pd.concat([data, datap2n], axis=0);
data2.shape

In [ ]:
data2= data2.rename(columns={'p1_liked':'liked'})
data2.liked.describe()

In [ ]:
datam=data2[(data2.p1_gender=='male') & (data2.p2_gender=='female')]
datam.shape

In [ ]:
dataf=data2[(data2.p1_gender=='female') & (data2.p2_gender=='male')]
dataf.shape

So the following rows will be ignored, and we will train the classifiers independantly for men vs women.

In [ ]:
data2[((data2.p1_gender!='male')|(data2.p2_gender!='female')) & ((data2.p1_gender!='female')|(data2.p2_gender!='male'))].shape

In [ ]:
datam.p1_gender.describe()

Removing more useless columns.

In [ ]:
useless_columns=['p1_gender', 'p2_gender', 'p2_liked', 'p1_id', 'p2_id']
datam.drop(useless_columns, axis=1, inplace=True)
dataf.drop(useless_columns, axis=1, inplace=True)

In [ ]:
print datam.shape
print dataf.shape

## Behavior analysis
Here is a rationale of separating the two data sets. Their behavior is completely different.

In [ ]:
dmvc = datam.liked.value_counts()
dfvc = dataf.liked.value_counts()
df = pd.DataFrame([dmvc,dfvc])
df.index = ['Males','Females']
df.plot(kind='bar', stacked=True);

In [ ]:
dmm = datam.liked.mean()
dfm = dataf.liked.mean()
df = pd.DataFrame([dmm,dfm])
df.index = ['Males','Females']
df.plot(kind='bar', stacked=True);

**Save** now , and let the analysis continue on separate data sets.

In [ ]:
datam.to_csv("1M-datam.csv", index=False)
dataf.to_csv("1M-dataf.csv", index=False)

In [ ]:
print datam.liked.sum(), "---", datam.liked.mean()
print dataf.liked.sum(), "---", dataf.liked.mean()

Males like 58.65% of the profiles shown to them, whereas females like only 5.7%. Even less than **one-tenth** compared to males.